In [1]:
#%autoreload 2
from data_loader import get_data,MXGenerator,get_dataset_means,normalize
from keras.applications import resnet50,xception,inception_v3,inception_resnet_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input as res_preprocess_input
from keras.applications.inception_v3 import preprocess_input as inc_preprocess_input
from keras.applications.xception import preprocess_input as xcep_preprocess_input
from keras.applications.inception_resnet_v2 import preprocess_input as inc_res_preprocess_input

import keras.layers as KL
import keras.models as KM
from keras.optimizers import *
import h5py
from keras.preprocessing.image import *
import numpy as np
from keras import regularizers

# classes count 
CLASS_COUNT = 997
# 读取文件名并且获取数据,因为数据集够大了,所以没有数据增强
train_x,valid_x,train_y,valid_y,test_images = get_data()

# 打乱数据集
# print(list(zip(train_x[:3],train_y[:3])))
train_set = list(zip(train_x[:3],train_y[:3]))
np.random.shuffle(train_set)
# print(train_set)
for i in range(len(train_set)):
    train_x[i] = train_set[i][0]
    train_y[i] = train_set[i][1]

# print(train_x,valid_x)
# 可以修改 32,64,128,256..在大就不好了
batch_size = 64

epochs = 15
# 得到结果的时候直接放入

train_path = np.concatenate([train_x,valid_x])
#print(train_path[:10])
#means,stds = get_dataset_means(train_path)

#print(means,stds)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# ResModel
Input = KL.Input(shape=(224,224,3))
# pre-processing
process_input = KL.Lambda(res_preprocess_input)(Input)
resnet_50 = resnet50.ResNet50(include_top=False, 
                              weights='imagenet', input_tensor=process_input, pooling='avg')
# trainable
for layer in resnet_50.layers:
    layer.trainable=False


# output_shape: batch_size * 2048
output = resnet_50.output
# dropout
# output = KL.Dropout(0.5)(output)
# classfier
output = KL.Dense(CLASS_COUNT,activation='softmax')(output) # ,kernel_regularizer=regularizers.l2(0.001)
# model
res_model = KM.Model(resnet_50.input,output)

In [6]:
# 会自动根据batch_size去生成数据,读取文件,以及normalize数据集,你可以读这些代码看
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(224,224),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(224,224),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
adam = Adam(lr=0.001)
# 
res_model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

res_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=1, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size))
                        
# 放开所有层
for layer in resnet_50.layers:
    layer.trainable=True                        
res_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

res_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=epochs - 5, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size))
                        
    res_model.save_weights("res_model.h5")

Epoch 1/1
8366/8366 [==============================] - 6327s 756ms/step - loss: 2.3948 - acc: 0.5192 - val_loss: 5.8971 - val_acc: 0.2162


In [7]:
### Training more
res_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=4, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size))

Epoch 1/4
2413/8366 [=======>......................] - ETA: 59:33 - loss: 1.6292 - acc: 0.6348

KeyboardInterrupt: 

In [ ]:
# save
res_model.save_weights("res_model_epoch_more.h5")

In [11]:
Input = KL.Input((299,299,3))
# pre-processing
process_input = KL.Lambda(inc_preprocess_input)(Input)

# InceptionV3
inception = inception_v3.InceptionV3(include_top=False, 
                              weights='imagenet', input_tensor=process_input, pooling='avg')
# train_able
for layer in inception.layers:
    layer.trainable = False

#output: (None, 2048)
output = inception.output
# dropout
output = KL.Dropout(0.5)(output)
# classfier
output = KL.Dense(CLASS_COUNT,activation='softmax',name='classfier')(output)
# model
inc_model = KM.Model(inception.input,output,name='inc_model')
inc_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 299, 299, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         lambda_5[0][0]                   
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation

In [12]:
# 会自动根据batch_size去生成数据,读取文件,以及normalize数据集,你可以读这些代码看
train_generator = MXGenerator((train_x,train_y),len(train_x), 
                              des_size=(299,299),means=None,stds=None, 
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x), 
                              des_size=(299,299),means=None,stds=None, 
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0) 

inc_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

inc_model.fit_generator(train_generator,len(train_x) // batch_size, 
                        epochs=1, verbose=1,validation_data=valid_generator, 
                        validation_steps=(len(valid_x) // batch_size)) 

# 放开所有层
# for layer in inception.layers:
#     layer.trainable = True
                        
# inc_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

# inc_model.fit_generator(train_generator,len(train_x) // batch_size,
#                     epochs=epochs-5, verbose=1,validation_data=valid_generator,
#                     validation_steps=(len(valid_x) // batch_size))
                        
inc_model.save_weights("inc_model.h5")

Epoch 1/1
1684/8366 [=====>........................] - ETA: 1:19:35 - loss: 4.4465 - acc: 0.2322

KeyboardInterrupt: 

In [10]:
# InceptionResNetV2
input_tensor = KL.Input(shape=(299,299,3))
# pre-processing,
process_input = KL.Lambda(normalize)(input_tensor)

inception_res = inception_resnet_v2.InceptionResNetV2(include_top=False, 
                              weights='imagenet', input_tensor=process_input, pooling='avg')
# train_able
for layer in inception_res.layers:
    layer.trainable = False
#output: (None, 1536)
output = inception_res.output
# dropout
output = KL.Dropout(0.5)(output)
# Dense
output = KL.Dense(2048,activation='relu')(output)
output = KL.Dropout(0.5)(output)
# classfier
output = KL.Dense(CLASS_COUNT,activation='softmax')(output)
# model
inc_res_model = KM.Model(inception_res.input,output,name='inc_res')

inc_res_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_392 (Conv2D)             (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_392 (BatchN (None, 111, 111, 32) 96          conv2d_392[0][0]                 
__________________________________________________________________________________________________
activation_392 (Activation)     (None, 111, 111, 32) 0           batch_normalization_392[0][0]    
__________________________________________________________________________________________________
conv2d_393

activation_542 (Activation)     (None, 12, 12, 160)  0           batch_normalization_542[0][0]    
__________________________________________________________________________________________________
conv2d_540 (Conv2D)             (None, 12, 12, 192)  208896      block17_18_ac[0][0]              
__________________________________________________________________________________________________
conv2d_543 (Conv2D)             (None, 12, 12, 192)  215040      activation_542[0][0]             
__________________________________________________________________________________________________
batch_normalization_540 (BatchN (None, 12, 12, 192)  576         conv2d_540[0][0]                 
__________________________________________________________________________________________________
batch_normalization_543 (BatchN (None, 12, 12, 192)  576         conv2d_543[0][0]                 
__________________________________________________________________________________________________
activation

In [ ]:
# 会自动根据batch_size去生成数据,读取文件,以及normalize数据集,你可以读这些代码看
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

inc_res_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

inc_res_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=1, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size))
inc_res_model.save_weights("inc_res_model.h5")

In [11]:
# xception
input_tensor = KL.Input(shape=(299,299,3))
# pre-processing
process_input = KL.Lambda(normalize)(input_tensor)
xcep = xception.Xception(include_top=False, 
                              weights='imagenet', input_tensor=process_input, pooling='avg')

# output: (None, 2048) 
output = xcep.output
# dropout
output = KL.Dropout(0.5)(output)
# classfier
output = KL.Dense(CLASS_COUNT,activation='softmax',name='classfier')(output)
# model
xcep_model = KM.Model(xcep.input,output,name='xception')

xcep_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [ ]:
# 会自动根据batch_size去生成数据,读取文件,以及normalize数据集,你可以读这些代码看
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

xcep_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

xcep_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=epochs, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size)
xcep_model.save_weights("xcep_model.h5")

### 提取特征向量

In [ ]:
# 挑选三个模型
# 导出特征向量
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)
#

xcep_feature_model = KM.Model(xcep_model.input,xcep_model.layers[-2].output)

xcep_train_feature_vector = xcep_feature_model.predict_generator(train_generator,steps=len(train_x) // batch_size)
xcep_valid_feature_vector = xcep_feature_model.predict_generator(valid_generator,steps=len(valid_x) // batch_size)
xcep_test_feature_vector = xcep_feature_model.predict_generator(test_generator,
                                                                steps=len(test_images) // batch_size)

# save to h5
xception_h5 = h5py.File("xception.h5",'w')
xception_h5.create_dataset('trian_x',
                           xcep_train_feature_vector.shape,data=xcep_train_feature_vector)
xception_h5.create_dataset('trian_label',
                           trian_y.shape,data=trian_y)
xception_h5.create_dataset('valid_x',
                           xcep_valid_feature_vector.shape,data=xcep_valid_feature_vector)
xception_h5.create_dataset('valid_label',
                           valid_y.shape,data=valid_y)
xception_h5.create_dataset('test',
                           xcep_test_feature_vector.shape,data=xcep_test_feature_vector)


In [12]:
## 第二个模型
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)
#

inc_res_feature_model = KM.Model(intputs=[inc_res_model.input],output=[inc_res_model.layers[-4].output])

inc_res_train_feature_vector = inc_res_feature_model.predict_generator(train_generator,steps=len(train_x) // batch_size)
inc_res_valid_feature_vector = inc_res_feature_model.predict_generator(valid_generator,steps=len(valid_x) // batch_size)
inc_res_test_feature_vector = inc_res_feature_model.predict_generator(test_generator,
                                                                steps=len(test_images) // batch_size)

# save to h5
inc_res_h5 = h5py.File("inc_res.h5",'w')
inc_res_h5.create_dataset('trian_x',
                           inc_res_train_feature_vector.shape,data=inc_res_train_feature_vector)
inc_res_h5.create_dataset('trian_label',
                           trian_y.shape,data=trian_y)
inc_res_h5.create_dataset('valid_x',
                           inc_res_valid_feature_vector.shape,data=inc_res_valid_feature_vector)
inc_res_h5.create_dataset('valid_label',
                           valid_y.shape,data=valid_y)
inc_res_h5.create_dataset('test',
                           inc_res_test_feature_vector.shape,data=inc_res_test_feature_vector)

In [ ]:
## 第三个模型
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)
#

res_feature_model = KM.Model(intputs=[res_model.input],output=[res_model.layers[-3].output])

res_train_feature_vector = res_feature_model.predict_generator(train_generator,
                                                                   steps=len(train_x) // batch_size)
res_valid_feature_vector = res_feature_model.predict_generator(valid_generator,
                                                                   steps=len(valid_x) // batch_size)
res_test_feature_vector = res_feature_model.predict_generator(test_generator,
                                                                steps=len(test_images) // batch_size)

# save to h5
res_h5 = h5py.File("res_.h5",'w')
res_h5.create_dataset('trian_x',
                           res_train_feature_vector.shape,data=res_train_feature_vector)
res_h5.create_dataset('trian_label',
                           trian_y.shape,data=trian_y)
res_h5.create_dataset('valid_x',
                           res_valid_feature_vector.shape,data=res_valid_feature_vector)
res_h5.create_dataset('valid_label',
                           valid_y.shape,data=valid_y)
res_h5.create_dataset('test',
                           res_test_feature_vector.shape,data=res_test_feature_vector)

In [ ]:
## 第三个模型
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)

res_feature_model = KM.Model(intputs=[res_model.input],output=[res_model.layers[-3].output])

res_train_feature_vector = res_feature_model.predict_generator(train_generator,
                                                                   steps=len(train_x) // batch_size)
res_valid_feature_vector = res_feature_model.predict_generator(valid_generator,
                                                                   steps=len(valid_x) // batch_size)
res_test_feature_vector = res_feature_model.predict_generator(test_generator,
                                                                steps=len(test_images) // batch_size)

# save to h5
res_h5 = h5py.File("res_.h5",'w')
res_h5.create_dataset('trian_x',
                           res_train_feature_vector.shape,data=res_train_feature_vector)
res_h5.create_dataset('trian_label',
                           trian_y.shape,data=trian_y)
res_h5.create_dataset('valid_x',
                           res_valid_feature_vector.shape,data=res_valid_feature_vector)
res_h5.create_dataset('valid_label',
                           valid_y.shape,data=valid_y)
res_h5.create_dataset('test',
                           res_test_feature_vector.shape,data=res_test_feature_vector)

In [ ]:
# inc 
train_generator = MXGenerator((train_x,train_y),len(train_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)

valid_generator = MXGenerator((valid_x,valid_y),len(valid_x),
                              des_size=(299,299),means=means,std=stds,
                              is_directory=True,batch_size=batch_size,shuffle=True,seed=0)
# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)
#

inc_feature_model = KM.Model(intputs=[inc_model.input],output=[inc_model.layers[-3].output])

inc_train_feature_vector = inc_feature_model.predict_generator(train_generator,
                                                                   steps=len(train_x) // batch_size)
inc_valid_feature_vector = inc_feature_model.predict_generator(valid_generator,
                                                                   steps=len(valid_x) // batch_size)
inc_test_feature_vector = inc_feature_model.predict_generator(test_generator,
                                                                steps=len(test_images) // batch_size)

# save to h5
inc_h5 = h5py.File("inc_.h5",'w')
inc_h5.create_dataset('trian_x',
                           inc_train_feature_vector.shape,data=inc_train_feature_vector)
inc_h5.create_dataset('trian_label',
                           trian_y.shape,data=trian_y)
inc_h5.create_dataset('valid_x',
                           inc_valid_feature_vector.shape,data=inc_valid_feature_vector)
inc_h5.create_dataset('valid_label',
                           valid_y.shape,data=valid_y)
inc_h5.create_dataset('test',
                           inc_test_feature_vector.shape,data=inc_test_feature_vector)

### 特征向量融合以及训练分类层

In [ ]:
X_train = []
y_trian = []
X_valid = []
y_valid = []
test = []

for file_name in ['xception.h5','inc_res.h5','res_.h5','inc_.h5']:
    with h5py.File(file_name,'r') as f:
        X_train.append(f['train_x'])
        X_valid.append(f['valid_x'])
        y_train.append(f['y_train'])
        y_valid.append(f['y_valid'])
        test.append(f['test'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid,axis=1)
test = np.concatenate(test,axis=1)
y_train = np.concatenate(y_train)
y_valid = np.concatenate(y_valid)

# build the network
merge_input = KL.Input(shape=X_train.shape[1:])
merge_dropout = KL.Dropout(0.5)(merge_input)
merge_classfier = KL.Dense(CLASS_COUNT,activation='softmax')(merge_dropout)
merge_model(inputs=[merge_input],outputs=[merge_classfier])

merge_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

datagen = ImageGenerator()

merge_train_generator = datagen.flow((X_train,y_train))
merge_valid_generator = datagen.flow((X_valid,y_valid))

merge_model.fit_generator(train_generator,len(train_x) // batch_size,
                    epochs=epochs, verbose=1,validation_data=valid_generator,
                    validation_steps=(len(valid_x) // batch_size)

### 输出预测结果

In [ ]:
import pandas as pd

df = pd.read_csv("kaggle_sample_submission.csv")

images = np.zeros((224,224,3),dtype=np.int)

for i in range(len(test_images)):
    img = cv2.imread(test_images[i])
    img = cv2.cvtColor(img,cv2.BGRA2RGB)
    img = cv2.resize(img,(224,224))
    images[i] = img

# test_generator
test_generator = MXGenerator((test_images,None),len(test_images),
                              des_size=(299,299),means=None,stds=None,
                              is_directory=True,batch_size=batch_size,shuffle=False,seed=0)

y_pred = res_model.predict_generator(test_generator)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'predict', y_pred[i])
    